**LearnPy: Python for Data Analytics**

- Author : Team Algoritma
- Developed by Algoritma's product division and instructors team
___

# Learn Python - Introduction to Machine Learning 2: Time Series & Forecasting

Selamat datang di Learn Python - Introduction to Machine Learning 2. Notebook ini berisi latihan bagaimana cara membuat model untuk melakukan forecasting pada data time series yang dapat digunakan untuk melatih kemampuan Anda. Pada bagian ini Anda akan diperkenalkan dengan workflow dari forecasting pada data time seris. Mari berlatih!

## Time Series & Forecasting

### Studi Kasus: Forecast Permintaan Barang pada Toko Retail Walmart

<p align="center">
<img src="assets/walmart.jpeg" alt="Walmart" width="650">
</p>

Walmart merupakan salah satu perusahaan Amerika Serikat yang mengelola jaringan toserba. Sebagai perusahaan yang bergerak dalam bidang toko retail/toserba, akan menjadi penting untuk selalu memastikan bahwa produk yang dijual selalu tersedia stoknya.

Untuk menyederhanakan operasinya, Walmart menggunakan machine learning untuk memperkirakan berapa banyak permintaan barang. Hal ini memerlukan prediksi penjualan di masa depan dengan meneliti data penjualan historis serta faktor eksternal seperti pola cuaca dan hari libur. Dengan membedakan pola dari penjualan sebelumnya, algoritma machine learning dapat menghasilkan prediksi yang tepat mengenai permintaan saham mereka di masa depan.

Jadi, mereka punya pertanyaan bisnis,

> "Bagaimana kami dapat memperkirakan permintaan produk di toko Walmart secara akurat, memanfaatkan data penjualan historis untuk mengoptimalkan manajemen inventaris dan memastikan ketersediaan produk?"

Melalui perkiraan permintaan yang tepat, tentu saja Walmart dapat menyesuaikan tingkat inventaris, mengatur promosi, dan mengoptimalkan staf untuk memenuhi kebutuhan pelanggan dengan baik.

Oleh karena itu, tugas kita sebagai data scientist adalah melakukan prediksi permintaan barang di masa depan berdasarkan data historis penjualan untuk memaksimalkan pelayanan terhadap pelanggan dan mendapatkan keuntungan semaksimal mungkin.

### 1. Load Data: Walmart Dataset

Mari kita mulai dengan mengimpor library yang diperlukan.

In [ ]:
# library untuk manipulasi data
import pandas as pd
import numpy as np

# library prophet untuk forecasting
from prophet import Prophet

# library untuk visualisasi data
import matplotlib.pyplot as plt

# libraryuntuk evaluasi model
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# library untuk mengabaikan warning terkait update library
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

# pandas output display setup
pd.options.display.float_format = '{:,.2f}'.format

Mari kita baca dataset `data_input/office_monthly_sales.csv`

In [ ]:
# memanggil dan membaca data
monthly_demand_train = pd.read_csv('data_input/office_monthly_sales.csv')
monthly_demand_train.head()

___
 🔎 **Deskripsi Data**

Dataset ini berisi informasi tentang penjualan bulanan, kuantitas, dan keuntungan untuk produk kategori **Perlengkapan Kantor** di toko ritel Walmart selama periode Januari 2020 hingga Desember 2022.

 🔎 **Penjelasan Kolom**
- `Order Month` $\rightarrow$ bulan dan tahun pemesanan barang dilakukan.
- `Total Sales` $\rightarrow$ total penjualan produk (USD) pada bulan dan tahun tersebut (laba kotor).
- `Total Quantity` $\rightarrow$ total produk yang terjual pada bulan dan tahun tersebut.
- `Total Profit` $\rightarrow$ total keuntungan yang diperoleh dari penjualan produk pada bulan dan tahun tersebut (laba bersih).
___

### 2. Exploratory Data Analysis (EDA)

#### 2.1 Cek Informasi Dasar

Mari kita pahami terlebih dahulu struktur dasar dan informasi tentang data menggunakan metode `.info()`. Metode ini memberikan wawasan berharga tentang DataFrame, termasuk tipe data, jumlah baris yang tidak nol, dan penggunaan memori.

In [ ]:
# cek informasi dasar
monthly_demand_train.info()

Dari fungsi `info()`, didapat beberapa informasi dasar tentang dataset:

- Dataset terdiri dari 36 data (baris).
- Terdapat 4 kolom.
- Tiap kolom memiliki 36 data bukan nol yang menandakan bahwa tidak ada nilai missing.
- Tipe data dari tiap kolom beragam. Perlu dilakukan proses lanjutan untuk mengubah ke tipe data yang sesuai.

#### 2.2 Cek Visualisasi

Salah satu aspek penting dalam analisis _Time Series_ adalah melakukan analisis eksplorasi visual karena membantu mengidentifikasi pola, tren, dan anomali dalam data secara intuitif, sehingga memudahkan dalam mengambil keputusan yang lebih tepat dan efektif. 

Mari kita visualisasikan berapa total penjualan tiap bulan

In [ ]:
# visualisasi produk terjual tiap bulan
plt.figure(figsize=(16, 3))
plt.plot(monthly_demand_train['Order Month'], monthly_demand_train['Total Quantity'], color='blue')
plt.title('Total Produk Terjual Tiap Bulan')
plt.xlabel('Bulan Pemesanan')
plt.ylabel('Total Terjual')
plt.xticks(rotation=45)
plt.show()

#### 2.3 Cek Data Statistik

Mari kita mulai dengan meninjau deskripsi statistik dari data kita. Melakukan deskripsi statistik adalah langkah penting untuk memahami karakteristik dasar dari setiap kolom dalam kumpulan data kita. Kita dapat menggunakan metode `.describe()`untuk mendapatkan gambaran umum mengenai distribusi dan variabilitas data kita sebelum melangkah ke tahap analisis yang lebih mendalam.

In [ ]:
# cek data statistik dataset
monthly_demand_train.describe()

___
**Soal 1.** Kapan total penjualan terbanyak terjadi?

> 🆘 Hint : Anda mungkin memerlukan fungsi `max`, lalu melakukan subsetting.

- [ ] Maret 2017
- [ ] Februari 2020
- [x] Desember 2022

<!-- 

Reference answer :

```python
# kode untuk menampilkan baris dengan total penjualan tertinggi
max_price = monthly_demand_train['Total Quantity'].max()
monthly_demand_train.loc[monthly_demand_train['Total Quantity'] == max_price]
```

Penjualan produk terbanyak terjadi pada bulan Desember tahun 2022

-->
___

In [ ]:
# your code here


___
**Soal 2.** Kapankah total penjualan terendah terjadi?

> 🆘 Hint : Anda mungkin memerlukan fungsi `min()`, lalu melakukan subsetting.

- [ ] Maret 2017
- [x] Februari 2020
- [ ] Desember 2022

<!-- 

Reference answer :

```python
# kode untuk menampilkan baris dengan total penjualan terendah
min_price = monthly_demand_train['Total Quantity'].min()
monthly_demand_train.loc[monthly_demand_train['Total Quantity'] == min_price]
```

Penjualan produk tersedikit/terendah terjadi pada bulan Februari tahun 2020

-->
___

In [ ]:
# your code here


### 3. Data Preprocessing

Untuk mengubah dataset menjadi data yang bersih dan sesuai agar menghasilkan performa model yang baik, maka dilakukan proses pembersihan data atau dikenal dengan **Data Preprocessing**.

#### 3.1 Memilih Kolom

Kasus ini akan melakukan forecasting terhadap jumlah pesanan/permitaan produk `TotalQuantity`, oleh karena itu kita ambil data `Order Month` dan `Total Quantity` saja.

In [ ]:
# memilih kolom yang akan digunakan untuk proses forecasting
monthly_demand_train = monthly_demand_train[['Order Month', 'Total Quantity']]
monthly_demand_train.head()

#### 3.2 Ubah Tipe Data

In [ ]:
# cek tipe data pada dataset monthly_demand_train
monthly_demand_train.dtypes

Terlihat kolom `Order Month` masih memiliki tipe data object. Ubahlah data dengan informasi tanggal ke dalam bentuk datetime menggunakan `pd.to_datetime()`. Setelah itu, urutkan data berdasarkan tanggalnya.

___
**Soal 3.** Berdasarkan informasi tanggal yang kita miliki, format tanggal seperti apa yang cocok?
  - [X] `%Y-%m-%d`
  - [ ] `%y-%m-%d`
  - [ ] `%y.%m.%d`

<!-- 

Reference answer :

```python
# mengubah tipe data
monthly_demand_train['Order Month'] = pd.to_datetime(monthly_demand_train['Order Month'], 
                                                     format="%Y-%m-%d")
```

Tipe yang cocok yaitu format %Y-%m-%d karena data kita berbentuk 2020-01-01 dengan tanda pemisah "-".

2020 -> Y
01 -> m
01 -> d

-->
___

In [ ]:
# mengubah tipe data
monthly_demand_train['Order Month'] = pd.to_datetime(monthly_demand_train['Order Month'], 
                                                     format= _____________ )

In [ ]:
# melakukan pengurutan kolom date secara ascending
monthly_demand_train = monthly_demand_train.sort_values('Order Month')
monthly_demand_train.head()

In [ ]:
# pemeriksaan ulang tipe data
monthly_demand_train.dtypes

### 4. Prophet Modeling

___
**Soal 4.** Metode perhitungan apa yang digunakan `prophet` untuk menggambarkan kondisi time series dan melakukan forecasting?

- [ ] _Component Decompose_
- [x] _General Additive Model_
- [ ] _Holt Winters Exponential Smoothing_

___

**Soal 5.** Berikut ini adalah kelebihan dari `prophet` yang tidak dimiliki oleh metode time series konvensional, kecuali ...

- [ ] fleksibilitas dalam mengatur komponen trend dan seasonal
- [x] melakukan forecasting masa depan yang sangat panjang
- [ ] menambahkan efek holiday/event
___


#### 3.1 Persiapan Data

`Prophet` hanya dapat menerima input data dengan judul kolom `ds` dan `y`, oleh karena itu ubahlah judul kolom pada data yang kita miliki.

In [ ]:
# mengubah nama kolom
monthly_demand_train = monthly_demand_train.rename(
    columns = {'Order Month': 'ds',
               'Total Quantity': 'y'}) 

# lihat data
monthly_demand_train.tail()

#### 3.2 _Fitting Model_

Setelah semua persiapan data selesai. Kini kita tugaskan `prophet` untuk mencari pola dari data time series kita. Silakan bangun sebuah model `prophet` tanpa menentukan parameter apa-apa (gunakan nilai defaultnya) setelah itu lakukan fitting kepada data `train`.

In [ ]:
# membuat object dari class Prophet
model_prophet = Prophet()

# proses fiting model
model_prophet.fit(monthly_demand_train)

#### 3.3 _Forecast_

Karena model kita telah belajar dari pola data masa lalu, kita akan coba meramal/*forecast* data kita untuk **1 tahun ke depan**. Untuk itu kita harus mempersipakan dataframe baru yang berisi informasi waktu yang ingin di-*forecast*. Hal ini dapat dilakukan menggunakan method `.make_future_dataframe()` dari objek model kita, yaitu `model_prophet`.

Parameter `.make_future_dataframe()`:
- `periods`: Banyaknya data yang ingin di-*forecast*
- `freq`: interval data. `D` (day), `M` (month), `MS` (month start), dan lainnya.

In [ ]:
# membuat data 1 tahun ke depan
future = model_prophet.make_future_dataframe(periods=12, freq='MS')

# cek data
future.tail()

In [ ]:
# proses forecast
forecast_result = model_prophet.predict(future)

# data hasil forecast
forecast_result.tail()

In [ ]:
# visualize
fig = model_prophet.plot(forecast_result)
plt.scatter(x = monthly_demand_train['ds'], y = monthly_demand_train['y'], color = 'red', s = 10);

#### 3.4 Interpretasi Model

Dalam melakukan interpretasi, dapat dilakukan dengan memvisualisasikan setiap komponennya menggunakan method `plot_components()` dari objek `model_prophet` dan memberikan input hasil forecast (`forecast`).

___
**Soal 6.** Berdasarkan visualisasi tiap komponen, manakah pernyataan yang benar?
  - [X] trend permintaan produk selalu meningkat
  - [ ] trend permintaan produk selalu turun
  - [ ] trend permintaan produk fluktuatif

<!-- 

Reference answer :

```python
# intepretasi model
fig = model_prophet.plot_components(forecast_result)
```
Berdasarkan plot, trend permintaan produk selalu meningkat

-->
___

In [ ]:
# intepretasi model
fig = model_prophet._____________

### 5. Evaluasi

Kami sebenarnya memiliki dua dataset pada folder `data_input`:
- Data train (`office_monthly_sales.csv`) digunakan untuk melatih model prophet untuk memperoleh pola dasar seperti trend dan seasonality.
- Data test (`test_office_monthly_sales.csv`) sengaja disimpan agar dapat melakukan __cross validation__ dan melihat bagaimana kinerja model yang telah kita miliki pada **unseed data** yaitu data baru yang belum pernah "dilihat" oleh model.

#### 4.1 Persiapan Data Test

Mari kita baca dataset `data_input/test_office_monthly_sales.csv`

In [ ]:
# memanggil dan membaca data
monthly_demand_test = pd.read_csv('data_input/test_office_monthly_sales.csv')
monthly_demand_test

Memilih kolom `Order Month` dan `Total Quality`, lalu ubah tipe `Order Month` ke bentuk datetime.

In [ ]:
# memilih kolom yang diperlukan dan rename kolom
monthly_demand_test = monthly_demand_test[['Order Month', 'Total Quantity']].rename(
    columns = {'Order Month': 'ds',
               'Total Quantity': 'y'}) 

# mengubah tipe data
monthly_demand_test['ds'] = pd.to_datetime(monthly_demand_test['ds'], 
                                           format="%Y-%m-%d")


# melihat data test
monthly_demand_test

#### 4.2 Evaluasi Model

Kami telah mengelompokkan data kami menggunakan tanggal pemisah (__cutoff__): `'1 Januari 2023'`. Mari kita definisikan ini sebagai variabel untuk memudahkan evaluasi model.

In [ ]:
# definisikan nilai cutoff
cutoff = '2023-01-01'

In [ ]:
# ambil data forecast setelah tanggal cutoff
forecast_test = forecast_result[forecast_result['ds'] >= cutoff]

# ambil hanya data ds dan yhat
forecast_test = forecast_test[['ds', 'yhat']]

# tampilkan hasil
forecast_test 

___
**Soal 7.** Berdasarkan nilai MAE pada data test, jika total permintaan produk sebanyak 500, berapa permintaan produk yang akan diprediksi oleh model kita?

- [x] 400 - 600
- [ ] 500 - 600
- [ ] 600 - 800

<!-- 

Reference answer :

```python
# Nilai MAE test
test_mae = mean_absolute_error(y_true = monthly_demand_test['y'],
                               y_pred = forecast_test['yhat'])

# tampilkan nilai MAE 
print(f"Nilai MAE Test: {test_mae:,.2f}")
```

Nilai MAE yang didapat yaitu 99.95 (dibulatkan menjadi 100). Maka hasil prediksi dari model kita berada di rentang (500 - MAE) sampai dengan (500 + MAE) -> 400 s.d. 600

-->
___

In [ ]:
# Nilai MAE test
test_mae = _____________(y_true = _____________ ,
                               y_pred = forecast_test['yhat'] )

# tampilkan nilai MAE 
print(f"Nilai MAE Test: {test_mae:,.2f}")

___
**Soal 8.** Besar rasio kesalahan prediksti total permintaan produk dari total permintaan yang sebenarnya?

- [ ] 10%
- [ ] 14%
- [x] 18%

<!-- 

Reference answer :

```python
# Nilai MAPE test
test_mape = mean_absolute_percentage_error(y_true = monthly_demand_test['y'],
                                           y_pred = forecast_test['yhat'])

# tampilkan nilai MAPE
print(f"Nilai MAPE Test: {test_mape * 100:,.2f}%")
```

Nilai MAPE yang didapat yaitu 18.22% (dibulatkan 18%). Nilai MAPE menginterpretasikan rasio kesalahan prediksti total permintaan produk dari total permintaan yang sebenarnya.

-->
___

In [ ]:
# Nilai MAPE test
test_mape = _____________(y_true = _____________ ,
                                           y_pred = forecast_test['yhat'])

# tampilkan nilai MAPE
print(f"Nilai MAPE Test: {test_mape * 100:,.2f}%")